In [40]:
import cv2
import numpy as np
import os
import scipy.io as sio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import csv
import time
import pickle

In [41]:
# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#CSV file reading
"""
Generating dictionaries
for photo_id ---> buisness_id
"""
photo_to_bus_dict = {}
with open('train_photo_to_biz_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            photo_to_bus_dict[row[0]] = row[1]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')
print(len(photo_to_bus_dict))
    
"""
Generating dictionaries
for business_id ---> labels
"""
bus_to_labels_dict = {}     
with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            bus_to_labels_dict[row[0]] = [row[1]]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')  
print(len(bus_to_labels_dict))

"""
Creates no_images*labels_size matrix
with each row indicating what labels
are assigned to each photo
"""
def id_to_label(photo_ids):
    photo_labels = np.empty(shape=[0,9])
    for id in photo_ids:
        lst_labels = bus_to_labels_dict[photo_to_bus_dict[id]]
        ls = lst_labels[0].split(' ')
        labels = [0]*9
        for label in ls:
            if label !='':
                labels[int(label)] = 1
        labels = np.array(labels).reshape(1,-1)
        photo_labels = np.append(photo_labels,labels,axis=0)
    return photo_labels


GPU is active
Column names are photo_id, business_id
Processed 234843 lines.
234842
Column names are business_id, labels
Processed 2001 lines.
2000


In [42]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model

#inputpath = 'gen_data/vgg7avg/'
inputpath = 'gen_data/vgg_resize/'
outpath = 'gen_data/vgg_outputs_app50k_'

random_state = np.random.RandomState(0)
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', C=1.0, probability=True, random_state=random_state, 
                                        max_iter = 10))
classifier_sgd = OneVsRestClassifier(linear_model.SGDClassifier(max_iter=10, tol=1e-3))
    
i = 1
file_list = sorted(os.listdir(inputpath))
train_list = file_list[:54000]
np_data = np.empty(shape=[0, 4096])
np_label = np.empty(shape=[0,9])

for file_name in train_list:
    print(file_name, '-', i)
    data = sio.loadmat(inputpath + file_name)
    data = data['features']
    #print(data.shape)
    f = file_name.split('.')
    label = id_to_label([f[0]])
    label = label.reshape((-1, 9))
    
    #classifier_sgd.partial_fit(data, label, classes=2)
    np_data = np.append(np_data, data, axis = 0)
    np_label = np.append(np_label, label, axis = 0)
    
    #print(label, label.shape)
    #break
    i = i + 1
 
sio.savemat(outpath + 'data' + '.vgg', {'data':np_data}, do_compression=True)
sio.savemat(outpath + 'label' + '.vgg', {'label':np_label}, do_compression=True)

#classifier.fit(np_data, np_label)    
    

10.jpg.vgg.mat - 1
100009.jpg.vgg.mat - 2
100021.jpg.vgg.mat - 3
100032.jpg.vgg.mat - 4
100034.jpg.vgg.mat - 5
100036.jpg.vgg.mat - 6
100039.jpg.vgg.mat - 7
10004.jpg.vgg.mat - 8
100043.jpg.vgg.mat - 9
100048.jpg.vgg.mat - 10
100061.jpg.vgg.mat - 11
100073.jpg.vgg.mat - 12
100087.jpg.vgg.mat - 13
100088.jpg.vgg.mat - 14
10009.jpg.vgg.mat - 15
100093.jpg.vgg.mat - 16
100097.jpg.vgg.mat - 17
1001.jpg.vgg.mat - 18
100100.jpg.vgg.mat - 19
100101.jpg.vgg.mat - 20
100105.jpg.vgg.mat - 21
100113.jpg.vgg.mat - 22
100135.jpg.vgg.mat - 23
10014.jpg.vgg.mat - 24
100147.jpg.vgg.mat - 25
10016.jpg.vgg.mat - 26
100163.jpg.vgg.mat - 27
100168.jpg.vgg.mat - 28
100179.jpg.vgg.mat - 29
100180.jpg.vgg.mat - 30
100181.jpg.vgg.mat - 31
100184.jpg.vgg.mat - 32
100185.jpg.vgg.mat - 33
1002.jpg.vgg.mat - 34
100206.jpg.vgg.mat - 35
100209.jpg.vgg.mat - 36
10022.jpg.vgg.mat - 37
100223.jpg.vgg.mat - 38
10023.jpg.vgg.mat - 39
100233.jpg.vgg.mat - 40
100235.jpg.vgg.mat - 41
100237.jpg.vgg.mat - 42
100239.jpg.vgg.

102120.jpg.vgg.mat - 364
102127.jpg.vgg.mat - 365
102128.jpg.vgg.mat - 366
102130.jpg.vgg.mat - 367
102131.jpg.vgg.mat - 368
102134.jpg.vgg.mat - 369
102138.jpg.vgg.mat - 370
102140.jpg.vgg.mat - 371
102149.jpg.vgg.mat - 372
102151.jpg.vgg.mat - 373
102154.jpg.vgg.mat - 374
10216.jpg.vgg.mat - 375
102170.jpg.vgg.mat - 376
102171.jpg.vgg.mat - 377
102176.jpg.vgg.mat - 378
10218.jpg.vgg.mat - 379
102180.jpg.vgg.mat - 380
102187.jpg.vgg.mat - 381
102188.jpg.vgg.mat - 382
102193.jpg.vgg.mat - 383
102199.jpg.vgg.mat - 384
102202.jpg.vgg.mat - 385
102203.jpg.vgg.mat - 386
102206.jpg.vgg.mat - 387
102210.jpg.vgg.mat - 388
102211.jpg.vgg.mat - 389
102213.jpg.vgg.mat - 390
102224.jpg.vgg.mat - 391
102227.jpg.vgg.mat - 392
102229.jpg.vgg.mat - 393
10223.jpg.vgg.mat - 394
102237.jpg.vgg.mat - 395
102249.jpg.vgg.mat - 396
102251.jpg.vgg.mat - 397
102256.jpg.vgg.mat - 398
102260.jpg.vgg.mat - 399
102269.jpg.vgg.mat - 400
102282.jpg.vgg.mat - 401
102298.jpg.vgg.mat - 402
102302.jpg.vgg.mat - 403
102

10418.jpg.vgg.mat - 718
104189.jpg.vgg.mat - 719
104192.jpg.vgg.mat - 720
104196.jpg.vgg.mat - 721
104205.jpg.vgg.mat - 722
104206.jpg.vgg.mat - 723
104208.jpg.vgg.mat - 724
104216.jpg.vgg.mat - 725
10422.jpg.vgg.mat - 726
104223.jpg.vgg.mat - 727
104228.jpg.vgg.mat - 728
104242.jpg.vgg.mat - 729
104254.jpg.vgg.mat - 730
104255.jpg.vgg.mat - 731
104262.jpg.vgg.mat - 732
104275.jpg.vgg.mat - 733
104277.jpg.vgg.mat - 734
104279.jpg.vgg.mat - 735
104285.jpg.vgg.mat - 736
104288.jpg.vgg.mat - 737
104291.jpg.vgg.mat - 738
104297.jpg.vgg.mat - 739
104313.jpg.vgg.mat - 740
104327.jpg.vgg.mat - 741
104331.jpg.vgg.mat - 742
104333.jpg.vgg.mat - 743
104336.jpg.vgg.mat - 744
104340.jpg.vgg.mat - 745
104348.jpg.vgg.mat - 746
104350.jpg.vgg.mat - 747
104354.jpg.vgg.mat - 748
104356.jpg.vgg.mat - 749
104357.jpg.vgg.mat - 750
104365.jpg.vgg.mat - 751
104366.jpg.vgg.mat - 752
104378.jpg.vgg.mat - 753
104387.jpg.vgg.mat - 754
104394.jpg.vgg.mat - 755
104398.jpg.vgg.mat - 756
104399.jpg.vgg.mat - 757
10

106082.jpg.vgg.mat - 1071
106094.jpg.vgg.mat - 1072
106107.jpg.vgg.mat - 1073
106110.jpg.vgg.mat - 1074
106117.jpg.vgg.mat - 1075
106119.jpg.vgg.mat - 1076
106120.jpg.vgg.mat - 1077
106125.jpg.vgg.mat - 1078
106126.jpg.vgg.mat - 1079
106134.jpg.vgg.mat - 1080
106135.jpg.vgg.mat - 1081
106136.jpg.vgg.mat - 1082
106140.jpg.vgg.mat - 1083
106145.jpg.vgg.mat - 1084
106146.jpg.vgg.mat - 1085
106147.jpg.vgg.mat - 1086
106148.jpg.vgg.mat - 1087
106153.jpg.vgg.mat - 1088
106164.jpg.vgg.mat - 1089
106170.jpg.vgg.mat - 1090
106171.jpg.vgg.mat - 1091
106173.jpg.vgg.mat - 1092
106185.jpg.vgg.mat - 1093
106186.jpg.vgg.mat - 1094
10620.jpg.vgg.mat - 1095
106201.jpg.vgg.mat - 1096
106211.jpg.vgg.mat - 1097
106213.jpg.vgg.mat - 1098
106214.jpg.vgg.mat - 1099
106218.jpg.vgg.mat - 1100
106227.jpg.vgg.mat - 1101
106232.jpg.vgg.mat - 1102
106237.jpg.vgg.mat - 1103
106240.jpg.vgg.mat - 1104
106247.jpg.vgg.mat - 1105
106257.jpg.vgg.mat - 1106
106270.jpg.vgg.mat - 1107
106286.jpg.vgg.mat - 1108
106289.jpg.vg

10793.jpg.vgg.mat - 1393
107931.jpg.vgg.mat - 1394
107936.jpg.vgg.mat - 1395
107938.jpg.vgg.mat - 1396
10794.jpg.vgg.mat - 1397
107955.jpg.vgg.mat - 1398
107956.jpg.vgg.mat - 1399
107957.jpg.vgg.mat - 1400
107967.jpg.vgg.mat - 1401
10797.jpg.vgg.mat - 1402
107970.jpg.vgg.mat - 1403
107972.jpg.vgg.mat - 1404
107978.jpg.vgg.mat - 1405
107979.jpg.vgg.mat - 1406
10798.jpg.vgg.mat - 1407
107983.jpg.vgg.mat - 1408
107992.jpg.vgg.mat - 1409
107999.jpg.vgg.mat - 1410
10801.jpg.vgg.mat - 1411
108012.jpg.vgg.mat - 1412
10802.jpg.vgg.mat - 1413
108022.jpg.vgg.mat - 1414
108023.jpg.vgg.mat - 1415
108026.jpg.vgg.mat - 1416
10803.jpg.vgg.mat - 1417
108030.jpg.vgg.mat - 1418
108031.jpg.vgg.mat - 1419
108049.jpg.vgg.mat - 1420
108061.jpg.vgg.mat - 1421
108064.jpg.vgg.mat - 1422
108067.jpg.vgg.mat - 1423
108070.jpg.vgg.mat - 1424
108085.jpg.vgg.mat - 1425
108091.jpg.vgg.mat - 1426
108094.jpg.vgg.mat - 1427
108097.jpg.vgg.mat - 1428
10810.jpg.vgg.mat - 1429
108106.jpg.vgg.mat - 1430
108112.jpg.vgg.mat -

...

401595.jpg.vgg.mat - 52493
401601.jpg.vgg.mat - 52494
401604.jpg.vgg.mat - 52495
401609.jpg.vgg.mat - 52496
401610.jpg.vgg.mat - 52497
401615.jpg.vgg.mat - 52498
401618.jpg.vgg.mat - 52499
401639.jpg.vgg.mat - 52500
401643.jpg.vgg.mat - 52501
401674.jpg.vgg.mat - 52502
401690.jpg.vgg.mat - 52503
401691.jpg.vgg.mat - 52504
401694.jpg.vgg.mat - 52505
401698.jpg.vgg.mat - 52506
401705.jpg.vgg.mat - 52507
401708.jpg.vgg.mat - 52508
401712.jpg.vgg.mat - 52509
401714.jpg.vgg.mat - 52510
401715.jpg.vgg.mat - 52511
401727.jpg.vgg.mat - 52512
401739.jpg.vgg.mat - 52513
401746.jpg.vgg.mat - 52514
40175.jpg.vgg.mat - 52515
401751.jpg.vgg.mat - 52516
401753.jpg.vgg.mat - 52517
401757.jpg.vgg.mat - 52518
401758.jpg.vgg.mat - 52519
401762.jpg.vgg.mat - 52520
401769.jpg.vgg.mat - 52521
401774.jpg.vgg.mat - 52522
401786.jpg.vgg.mat - 52523
401789.jpg.vgg.mat - 52524
401800.jpg.vgg.mat - 52525
401801.jpg.vgg.mat - 52526
401802.jpg.vgg.mat - 52527
401814.jpg.vgg.mat - 52528
401823.jpg.vgg.mat - 52529
40

403624.jpg.vgg.mat - 52798
403629.jpg.vgg.mat - 52799
403634.jpg.vgg.mat - 52800
403638.jpg.vgg.mat - 52801
403641.jpg.vgg.mat - 52802
403643.jpg.vgg.mat - 52803
403661.jpg.vgg.mat - 52804
403664.jpg.vgg.mat - 52805
403676.jpg.vgg.mat - 52806
40368.jpg.vgg.mat - 52807
403681.jpg.vgg.mat - 52808
403689.jpg.vgg.mat - 52809
403694.jpg.vgg.mat - 52810
403697.jpg.vgg.mat - 52811
403699.jpg.vgg.mat - 52812
4037.jpg.vgg.mat - 52813
403718.jpg.vgg.mat - 52814
403719.jpg.vgg.mat - 52815
403726.jpg.vgg.mat - 52816
403727.jpg.vgg.mat - 52817
403735.jpg.vgg.mat - 52818
403739.jpg.vgg.mat - 52819
403740.jpg.vgg.mat - 52820
403742.jpg.vgg.mat - 52821
403746.jpg.vgg.mat - 52822
403755.jpg.vgg.mat - 52823
40376.jpg.vgg.mat - 52824
403761.jpg.vgg.mat - 52825
403769.jpg.vgg.mat - 52826
403781.jpg.vgg.mat - 52827
403782.jpg.vgg.mat - 52828
40379.jpg.vgg.mat - 52829
403798.jpg.vgg.mat - 52830
4038.jpg.vgg.mat - 52831
403805.jpg.vgg.mat - 52832
403806.jpg.vgg.mat - 52833
403810.jpg.vgg.mat - 52834
403812.j

405376.jpg.vgg.mat - 53103
405378.jpg.vgg.mat - 53104
405381.jpg.vgg.mat - 53105
405395.jpg.vgg.mat - 53106
405397.jpg.vgg.mat - 53107
405404.jpg.vgg.mat - 53108
405419.jpg.vgg.mat - 53109
40542.jpg.vgg.mat - 53110
405429.jpg.vgg.mat - 53111
405431.jpg.vgg.mat - 53112
405433.jpg.vgg.mat - 53113
405437.jpg.vgg.mat - 53114
40544.jpg.vgg.mat - 53115
405448.jpg.vgg.mat - 53116
405452.jpg.vgg.mat - 53117
405453.jpg.vgg.mat - 53118
405466.jpg.vgg.mat - 53119
405476.jpg.vgg.mat - 53120
405487.jpg.vgg.mat - 53121
405498.jpg.vgg.mat - 53122
405500.jpg.vgg.mat - 53123
405502.jpg.vgg.mat - 53124
405522.jpg.vgg.mat - 53125
405536.jpg.vgg.mat - 53126
405558.jpg.vgg.mat - 53127
40556.jpg.vgg.mat - 53128
405567.jpg.vgg.mat - 53129
405569.jpg.vgg.mat - 53130
405571.jpg.vgg.mat - 53131
405572.jpg.vgg.mat - 53132
405577.jpg.vgg.mat - 53133
405578.jpg.vgg.mat - 53134
405586.jpg.vgg.mat - 53135
40559.jpg.vgg.mat - 53136
4056.jpg.vgg.mat - 53137
405603.jpg.vgg.mat - 53138
405617.jpg.vgg.mat - 53139
405619.

407251.jpg.vgg.mat - 53408
407254.jpg.vgg.mat - 53409
407255.jpg.vgg.mat - 53410
407258.jpg.vgg.mat - 53411
407275.jpg.vgg.mat - 53412
407289.jpg.vgg.mat - 53413
407290.jpg.vgg.mat - 53414
407301.jpg.vgg.mat - 53415
407303.jpg.vgg.mat - 53416
407307.jpg.vgg.mat - 53417
407308.jpg.vgg.mat - 53418
407312.jpg.vgg.mat - 53419
407322.jpg.vgg.mat - 53420
407325.jpg.vgg.mat - 53421
407336.jpg.vgg.mat - 53422
40734.jpg.vgg.mat - 53423
407340.jpg.vgg.mat - 53424
407349.jpg.vgg.mat - 53425
407356.jpg.vgg.mat - 53426
407361.jpg.vgg.mat - 53427
407366.jpg.vgg.mat - 53428
40738.jpg.vgg.mat - 53429
407385.jpg.vgg.mat - 53430
407389.jpg.vgg.mat - 53431
407390.jpg.vgg.mat - 53432
407391.jpg.vgg.mat - 53433
407399.jpg.vgg.mat - 53434
407403.jpg.vgg.mat - 53435
407410.jpg.vgg.mat - 53436
407418.jpg.vgg.mat - 53437
407425.jpg.vgg.mat - 53438
407426.jpg.vgg.mat - 53439
407429.jpg.vgg.mat - 53440
40743.jpg.vgg.mat - 53441
407434.jpg.vgg.mat - 53442
407439.jpg.vgg.mat - 53443
407443.jpg.vgg.mat - 53444
4074

409004.jpg.vgg.mat - 53713
409012.jpg.vgg.mat - 53714
409021.jpg.vgg.mat - 53715
409030.jpg.vgg.mat - 53716
409031.jpg.vgg.mat - 53717
409040.jpg.vgg.mat - 53718
409045.jpg.vgg.mat - 53719
409053.jpg.vgg.mat - 53720
409055.jpg.vgg.mat - 53721
409064.jpg.vgg.mat - 53722
409068.jpg.vgg.mat - 53723
409076.jpg.vgg.mat - 53724
409095.jpg.vgg.mat - 53725
409096.jpg.vgg.mat - 53726
409100.jpg.vgg.mat - 53727
409105.jpg.vgg.mat - 53728
409107.jpg.vgg.mat - 53729
409112.jpg.vgg.mat - 53730
409117.jpg.vgg.mat - 53731
409123.jpg.vgg.mat - 53732
409145.jpg.vgg.mat - 53733
409146.jpg.vgg.mat - 53734
409147.jpg.vgg.mat - 53735
409153.jpg.vgg.mat - 53736
409172.jpg.vgg.mat - 53737
40918.jpg.vgg.mat - 53738
409187.jpg.vgg.mat - 53739
409188.jpg.vgg.mat - 53740
409194.jpg.vgg.mat - 53741
409198.jpg.vgg.mat - 53742
409204.jpg.vgg.mat - 53743
409205.jpg.vgg.mat - 53744
409207.jpg.vgg.mat - 53745
409213.jpg.vgg.mat - 53746
409220.jpg.vgg.mat - 53747
409222.jpg.vgg.mat - 53748
409226.jpg.vgg.mat - 53749
40

In [31]:
np_data = sio.loadmat('gen_data/vgg_outputs_10k_data.vgg')
np_data = np_data['data']
np_label = sio.loadmat('gen_data/vgg_outputs_10k_label.vgg')
np_label = np_label['label']

classifier_2 = OneVsRestClassifier(svm.SVC(kernel='linear', C=1.0, probability=True, random_state=random_state, 
                                        max_iter = 100))

print("Training start")
t1 = time.time()
classifier_2.fit(np_data, np_label)
print("Training done")
print("Training time for 50000 images took %0.3fs." % (time.time() - t1))


Training start


/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarni

Training done
Training time for 50000 images took 3750.127s.


/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [43]:
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.svm import LinearSVC
"""
np_data = sio.loadmat('gen_data/vgg_outputs_10k_data.vgg')
np_data = np_data['data']
np_label = sio.loadmat('gen_data/vgg_outputs_10k_label.vgg')
np_label = np_label['label']
"""

print("Training start")
t1 = time.time()
classifier_3 = MultiOutputClassifier(LinearSVC(random_state=0, max_iter=100))
classifier_3.fit(np_data, np_label)

print("Training done")
print("Training time of MultiOutputClassifier for 50000 images took %0.3fs." % (time.time() - t1))

Training start
Training done
Training time of MultiOutputClassifier for 50000 images took 636.348s.


In [44]:
### testing the model
#inputpath = 'gen_data/vgg7avg/'
inputpath = 'gen_data/correct_test/vgg_resize/'

test_list = sorted(os.listdir(inputpath))
pred_list = []
label_list = []
i = 1
for file_name in test_list:
    print(file_name, '-', i)
    data = sio.loadmat(inputpath + file_name)
    data = data['features']
    #print(data.shape)
    f = file_name.split('.')
    label = id_to_label([f[0]])
    label = label.reshape((-1, 9))
    #print(label)
    #break
    pred = classifier_3.predict(data)
    i = i + 1
    label_list.append(label)
    pred_list.append(pred)

300489.jpg.vgg.mat - 1
300490.jpg.vgg.mat - 2
3005.jpg.vgg.mat - 3
30050.jpg.vgg.mat - 4
30051.jpg.vgg.mat - 5
300512.jpg.vgg.mat - 6
300523.jpg.vgg.mat - 7
300534.jpg.vgg.mat - 8
300543.jpg.vgg.mat - 9
300547.jpg.vgg.mat - 10
300552.jpg.vgg.mat - 11
300554.jpg.vgg.mat - 12
300556.jpg.vgg.mat - 13
300560.jpg.vgg.mat - 14
300563.jpg.vgg.mat - 15
300568.jpg.vgg.mat - 16
300569.jpg.vgg.mat - 17
300570.jpg.vgg.mat - 18
300571.jpg.vgg.mat - 19
300576.jpg.vgg.mat - 20
300585.jpg.vgg.mat - 21
300587.jpg.vgg.mat - 22
300593.jpg.vgg.mat - 23
300609.jpg.vgg.mat - 24
30062.jpg.vgg.mat - 25
300626.jpg.vgg.mat - 26
300634.jpg.vgg.mat - 27
300640.jpg.vgg.mat - 28
300642.jpg.vgg.mat - 29
300643.jpg.vgg.mat - 30
300644.jpg.vgg.mat - 31
300645.jpg.vgg.mat - 32
300654.jpg.vgg.mat - 33
300658.jpg.vgg.mat - 34
300660.jpg.vgg.mat - 35
300664.jpg.vgg.mat - 36
300668.jpg.vgg.mat - 37
300674.jpg.vgg.mat - 38
300682.jpg.vgg.mat - 39
300689.jpg.vgg.mat - 40
300701.jpg.vgg.mat - 41
300717.jpg.vgg.mat - 42
300718

302741.jpg.vgg.mat - 397
302748.jpg.vgg.mat - 398
302755.jpg.vgg.mat - 399
302764.jpg.vgg.mat - 400
30277.jpg.vgg.mat - 401
302784.jpg.vgg.mat - 402
302789.jpg.vgg.mat - 403
302790.jpg.vgg.mat - 404
302792.jpg.vgg.mat - 405
302802.jpg.vgg.mat - 406
302816.jpg.vgg.mat - 407
302829.jpg.vgg.mat - 408
302835.jpg.vgg.mat - 409
302838.jpg.vgg.mat - 410
302843.jpg.vgg.mat - 411
302856.jpg.vgg.mat - 412
302857.jpg.vgg.mat - 413
302859.jpg.vgg.mat - 414
302868.jpg.vgg.mat - 415
302869.jpg.vgg.mat - 416
302873.jpg.vgg.mat - 417
302875.jpg.vgg.mat - 418
302878.jpg.vgg.mat - 419
302888.jpg.vgg.mat - 420
302890.jpg.vgg.mat - 421
302905.jpg.vgg.mat - 422
302909.jpg.vgg.mat - 423
302913.jpg.vgg.mat - 424
302918.jpg.vgg.mat - 425
302929.jpg.vgg.mat - 426
302956.jpg.vgg.mat - 427
302961.jpg.vgg.mat - 428
302962.jpg.vgg.mat - 429
302965.jpg.vgg.mat - 430
302969.jpg.vgg.mat - 431
302977.jpg.vgg.mat - 432
302982.jpg.vgg.mat - 433
302987.jpg.vgg.mat - 434
302990.jpg.vgg.mat - 435
302998.jpg.vgg.mat - 436
3

305172.jpg.vgg.mat - 812
305173.jpg.vgg.mat - 813
305175.jpg.vgg.mat - 814
305181.jpg.vgg.mat - 815
305182.jpg.vgg.mat - 816
305189.jpg.vgg.mat - 817
305193.jpg.vgg.mat - 818
30520.jpg.vgg.mat - 819
305202.jpg.vgg.mat - 820
305208.jpg.vgg.mat - 821
305210.jpg.vgg.mat - 822
305212.jpg.vgg.mat - 823
305213.jpg.vgg.mat - 824
305218.jpg.vgg.mat - 825
305222.jpg.vgg.mat - 826
305231.jpg.vgg.mat - 827
305239.jpg.vgg.mat - 828
30525.jpg.vgg.mat - 829
305255.jpg.vgg.mat - 830
305258.jpg.vgg.mat - 831
305261.jpg.vgg.mat - 832
305265.jpg.vgg.mat - 833
305266.jpg.vgg.mat - 834
305269.jpg.vgg.mat - 835
305285.jpg.vgg.mat - 836
305290.jpg.vgg.mat - 837
305293.jpg.vgg.mat - 838
305303.jpg.vgg.mat - 839
305306.jpg.vgg.mat - 840
305337.jpg.vgg.mat - 841
305339.jpg.vgg.mat - 842
305347.jpg.vgg.mat - 843
305358.jpg.vgg.mat - 844
305373.jpg.vgg.mat - 845
305380.jpg.vgg.mat - 846
305381.jpg.vgg.mat - 847
305384.jpg.vgg.mat - 848
305400.jpg.vgg.mat - 849
305402.jpg.vgg.mat - 850
305407.jpg.vgg.mat - 851
30

30774.jpg.vgg.mat - 1221
30775.jpg.vgg.mat - 1222
307754.jpg.vgg.mat - 1223
307761.jpg.vgg.mat - 1224
307769.jpg.vgg.mat - 1225
307787.jpg.vgg.mat - 1226
307790.jpg.vgg.mat - 1227
307809.jpg.vgg.mat - 1228
307810.jpg.vgg.mat - 1229
307815.jpg.vgg.mat - 1230
307817.jpg.vgg.mat - 1231
307822.jpg.vgg.mat - 1232
307834.jpg.vgg.mat - 1233
30784.jpg.vgg.mat - 1234
307844.jpg.vgg.mat - 1235
307845.jpg.vgg.mat - 1236
307847.jpg.vgg.mat - 1237
30786.jpg.vgg.mat - 1238
307866.jpg.vgg.mat - 1239
307871.jpg.vgg.mat - 1240
307880.jpg.vgg.mat - 1241
30789.jpg.vgg.mat - 1242
307898.jpg.vgg.mat - 1243
3079.jpg.vgg.mat - 1244
307906.jpg.vgg.mat - 1245
307916.jpg.vgg.mat - 1246
307919.jpg.vgg.mat - 1247
307922.jpg.vgg.mat - 1248
307924.jpg.vgg.mat - 1249
307937.jpg.vgg.mat - 1250
307940.jpg.vgg.mat - 1251
307941.jpg.vgg.mat - 1252
307943.jpg.vgg.mat - 1253
30796.jpg.vgg.mat - 1254
307961.jpg.vgg.mat - 1255
307962.jpg.vgg.mat - 1256
307964.jpg.vgg.mat - 1257
307969.jpg.vgg.mat - 1258
307980.jpg.vgg.mat -

310259.jpg.vgg.mat - 1632
310260.jpg.vgg.mat - 1633
310261.jpg.vgg.mat - 1634
310264.jpg.vgg.mat - 1635
310271.jpg.vgg.mat - 1636
310272.jpg.vgg.mat - 1637
310273.jpg.vgg.mat - 1638
310275.jpg.vgg.mat - 1639
310285.jpg.vgg.mat - 1640
310288.jpg.vgg.mat - 1641
310291.jpg.vgg.mat - 1642
310294.jpg.vgg.mat - 1643
310301.jpg.vgg.mat - 1644
310304.jpg.vgg.mat - 1645
310305.jpg.vgg.mat - 1646
310307.jpg.vgg.mat - 1647
310312.jpg.vgg.mat - 1648
310313.jpg.vgg.mat - 1649
310316.jpg.vgg.mat - 1650
310321.jpg.vgg.mat - 1651
310322.jpg.vgg.mat - 1652
310326.jpg.vgg.mat - 1653
310329.jpg.vgg.mat - 1654
31033.jpg.vgg.mat - 1655
310346.jpg.vgg.mat - 1656
31035.jpg.vgg.mat - 1657
310350.jpg.vgg.mat - 1658
310361.jpg.vgg.mat - 1659
310386.jpg.vgg.mat - 1660
310387.jpg.vgg.mat - 1661
310396.jpg.vgg.mat - 1662
310400.jpg.vgg.mat - 1663
310404.jpg.vgg.mat - 1664
310410.jpg.vgg.mat - 1665
310415.jpg.vgg.mat - 1666
310416.jpg.vgg.mat - 1667
310430.jpg.vgg.mat - 1668
310441.jpg.vgg.mat - 1669
310443.jpg.vgg

...

8931.jpg.vgg.mat - 37313
89315.jpg.vgg.mat - 37314
89325.jpg.vgg.mat - 37315
89327.jpg.vgg.mat - 37316
89331.jpg.vgg.mat - 37317
89339.jpg.vgg.mat - 37318
8934.jpg.vgg.mat - 37319
89348.jpg.vgg.mat - 37320
89350.jpg.vgg.mat - 37321
89353.jpg.vgg.mat - 37322
89363.jpg.vgg.mat - 37323
89368.jpg.vgg.mat - 37324
89370.jpg.vgg.mat - 37325
89374.jpg.vgg.mat - 37326
8938.jpg.vgg.mat - 37327
89382.jpg.vgg.mat - 37328
89388.jpg.vgg.mat - 37329
89390.jpg.vgg.mat - 37330
8940.jpg.vgg.mat - 37331
89413.jpg.vgg.mat - 37332
89431.jpg.vgg.mat - 37333
89432.jpg.vgg.mat - 37334
89437.jpg.vgg.mat - 37335
8946.jpg.vgg.mat - 37336
89472.jpg.vgg.mat - 37337
89480.jpg.vgg.mat - 37338
89486.jpg.vgg.mat - 37339
89490.jpg.vgg.mat - 37340
89497.jpg.vgg.mat - 37341
89499.jpg.vgg.mat - 37342
89503.jpg.vgg.mat - 37343
8951.jpg.vgg.mat - 37344
89511.jpg.vgg.mat - 37345
89512.jpg.vgg.mat - 37346
89517.jpg.vgg.mat - 37347
89521.jpg.vgg.mat - 37348
89523.jpg.vgg.mat - 37349
8953.jpg.vgg.mat - 37350
89533.jpg.vgg.mat -

91690.jpg.vgg.mat - 37722
91691.jpg.vgg.mat - 37723
91696.jpg.vgg.mat - 37724
91716.jpg.vgg.mat - 37725
91723.jpg.vgg.mat - 37726
91724.jpg.vgg.mat - 37727
91736.jpg.vgg.mat - 37728
91737.jpg.vgg.mat - 37729
91749.jpg.vgg.mat - 37730
91756.jpg.vgg.mat - 37731
91767.jpg.vgg.mat - 37732
91771.jpg.vgg.mat - 37733
91774.jpg.vgg.mat - 37734
91775.jpg.vgg.mat - 37735
91783.jpg.vgg.mat - 37736
9180.jpg.vgg.mat - 37737
91801.jpg.vgg.mat - 37738
91814.jpg.vgg.mat - 37739
91816.jpg.vgg.mat - 37740
91823.jpg.vgg.mat - 37741
91825.jpg.vgg.mat - 37742
91828.jpg.vgg.mat - 37743
9185.jpg.vgg.mat - 37744
91853.jpg.vgg.mat - 37745
91869.jpg.vgg.mat - 37746
91874.jpg.vgg.mat - 37747
91878.jpg.vgg.mat - 37748
91882.jpg.vgg.mat - 37749
91883.jpg.vgg.mat - 37750
91892.jpg.vgg.mat - 37751
91901.jpg.vgg.mat - 37752
91909.jpg.vgg.mat - 37753
91910.jpg.vgg.mat - 37754
91912.jpg.vgg.mat - 37755
91915.jpg.vgg.mat - 37756
91917.jpg.vgg.mat - 37757
91918.jpg.vgg.mat - 37758
9192.jpg.vgg.mat - 37759
91921.jpg.vgg.m

94128.jpg.vgg.mat - 38142
94131.jpg.vgg.mat - 38143
94139.jpg.vgg.mat - 38144
94142.jpg.vgg.mat - 38145
94148.jpg.vgg.mat - 38146
94150.jpg.vgg.mat - 38147
94156.jpg.vgg.mat - 38148
94164.jpg.vgg.mat - 38149
94165.jpg.vgg.mat - 38150
9417.jpg.vgg.mat - 38151
94170.jpg.vgg.mat - 38152
94198.jpg.vgg.mat - 38153
942.jpg.vgg.mat - 38154
94201.jpg.vgg.mat - 38155
94202.jpg.vgg.mat - 38156
94203.jpg.vgg.mat - 38157
94211.jpg.vgg.mat - 38158
94223.jpg.vgg.mat - 38159
94228.jpg.vgg.mat - 38160
94229.jpg.vgg.mat - 38161
94237.jpg.vgg.mat - 38162
94239.jpg.vgg.mat - 38163
94244.jpg.vgg.mat - 38164
94245.jpg.vgg.mat - 38165
94251.jpg.vgg.mat - 38166
94256.jpg.vgg.mat - 38167
94264.jpg.vgg.mat - 38168
94271.jpg.vgg.mat - 38169
94278.jpg.vgg.mat - 38170
94287.jpg.vgg.mat - 38171
94310.jpg.vgg.mat - 38172
9432.jpg.vgg.mat - 38173
94340.jpg.vgg.mat - 38174
94345.jpg.vgg.mat - 38175
9436.jpg.vgg.mat - 38176
94360.jpg.vgg.mat - 38177
94362.jpg.vgg.mat - 38178
94367.jpg.vgg.mat - 38179
94378.jpg.vgg.mat

96608.jpg.vgg.mat - 38554
96615.jpg.vgg.mat - 38555
96640.jpg.vgg.mat - 38556
96671.jpg.vgg.mat - 38557
9668.jpg.vgg.mat - 38558
96710.jpg.vgg.mat - 38559
96712.jpg.vgg.mat - 38560
96719.jpg.vgg.mat - 38561
96721.jpg.vgg.mat - 38562
96726.jpg.vgg.mat - 38563
96729.jpg.vgg.mat - 38564
96737.jpg.vgg.mat - 38565
96738.jpg.vgg.mat - 38566
96741.jpg.vgg.mat - 38567
96748.jpg.vgg.mat - 38568
96751.jpg.vgg.mat - 38569
96764.jpg.vgg.mat - 38570
96767.jpg.vgg.mat - 38571
96768.jpg.vgg.mat - 38572
9677.jpg.vgg.mat - 38573
96774.jpg.vgg.mat - 38574
96787.jpg.vgg.mat - 38575
96788.jpg.vgg.mat - 38576
96789.jpg.vgg.mat - 38577
9679.jpg.vgg.mat - 38578
96790.jpg.vgg.mat - 38579
96801.jpg.vgg.mat - 38580
96803.jpg.vgg.mat - 38581
96806.jpg.vgg.mat - 38582
96816.jpg.vgg.mat - 38583
9682.jpg.vgg.mat - 38584
96823.jpg.vgg.mat - 38585
9683.jpg.vgg.mat - 38586
96834.jpg.vgg.mat - 38587
96836.jpg.vgg.mat - 38588
96841.jpg.vgg.mat - 38589
96842.jpg.vgg.mat - 38590
96852.jpg.vgg.mat - 38591
96857.jpg.vgg.mat

99071.jpg.vgg.mat - 38968
99074.jpg.vgg.mat - 38969
99075.jpg.vgg.mat - 38970
99088.jpg.vgg.mat - 38971
99093.jpg.vgg.mat - 38972
99094.jpg.vgg.mat - 38973
99102.jpg.vgg.mat - 38974
9911.jpg.vgg.mat - 38975
99112.jpg.vgg.mat - 38976
99113.jpg.vgg.mat - 38977
99114.jpg.vgg.mat - 38978
99121.jpg.vgg.mat - 38979
99130.jpg.vgg.mat - 38980
99140.jpg.vgg.mat - 38981
99143.jpg.vgg.mat - 38982
99148.jpg.vgg.mat - 38983
99151.jpg.vgg.mat - 38984
99152.jpg.vgg.mat - 38985
99167.jpg.vgg.mat - 38986
99187.jpg.vgg.mat - 38987
9919.jpg.vgg.mat - 38988
99195.jpg.vgg.mat - 38989
99196.jpg.vgg.mat - 38990
99213.jpg.vgg.mat - 38991
99215.jpg.vgg.mat - 38992
99224.jpg.vgg.mat - 38993
99226.jpg.vgg.mat - 38994
99238.jpg.vgg.mat - 38995
99259.jpg.vgg.mat - 38996
99264.jpg.vgg.mat - 38997
99267.jpg.vgg.mat - 38998
99271.jpg.vgg.mat - 38999
99277.jpg.vgg.mat - 39000
99279.jpg.vgg.mat - 39001
9928.jpg.vgg.mat - 39002
99288.jpg.vgg.mat - 39003
9929.jpg.vgg.mat - 39004
993.jpg.vgg.mat - 39005
99301.jpg.vgg.mat 

In [46]:
filename = 'vgg_multi_svc_model.sav'
pickle.dump(classifier_3, open(filename, 'wb'))

def verify_correctness(pred1, test_labels, threshold=0):
    cnt=0
    i = 1
    for x, y in zip(pred1, test_labels):
        #x1= np.array(x)
        #y1 = np.array(y)
        #print(x, y)
        #break
        c=0
        for a, b in zip(x[0], y[0]):
            #print(a, b)
            #break
            if(a!=b): c+=1
        if c<=threshold:
            cnt+=1
        #if i > 50 and i < 100:
            #print(i,'-',cnt,'-', x1, y1)
        i = i + 1
    accuracy = cnt/len(test_labels)
    return accuracy

#for p in pred_list[:50]:
#    print(p)
#print(label_list[0].shape)
#print(pred_list[1].shape, label_list[1].shape)
#pred_list = np.asarray(pred_list).reshape((-1, 9))
#label_list = np.asarray(label_list).reshape(-1, 9)

#print(pred_list.shape, label_list.shape)

print("Accuracy score for strict match is ", verify_correctness(pred_list, label_list, 0)*100)
print("Accuracy score for maximum of 1 mistmatch is ", verify_correctness(pred_list, label_list, 1)*100)
print("Accuracy score for maximum of 2 mistmatch is ", verify_correctness(pred_list, label_list, 2)*100)
print("Accuracy score for maximum of 3 mistmatch is ", verify_correctness(pred_list, label_list, 3)*100)
print("Accuracy score for maximum of 4 mistmatch is ", verify_correctness(pred_list, label_list, 4)*100)
print("Accuracy score for maximum of 5 mistmatch is ", verify_correctness(pred_list, label_list, 5)*100)
print("Accuracy score for maximum of 6 mistmatch is ", verify_correctness(pred_list, label_list, 6)*100)
print("Accuracy score for maximum of 7 mistmatch is ", verify_correctness(pred_list, label_list, 7)*100)
print("Accuracy score for maximum of 8 mistmatch is ", verify_correctness(pred_list, label_list, 8)*100)
print("Accuracy score for maximum of 9 mistmatch is ", verify_correctness(pred_list, label_list, 9)*100)

    

Accuracy score for strict match is  0.6311002095150493
Accuracy score for maximum of 1 mistmatch is  4.220961725177577
Accuracy score for maximum of 2 mistmatch is  15.016096887935
Accuracy score for maximum of 3 mistmatch is  38.589094997189434
Accuracy score for maximum of 4 mistmatch is  72.12427819510451
Accuracy score for maximum of 5 mistmatch is  92.71807450559558
Accuracy score for maximum of 6 mistmatch is  98.36476059073024
Accuracy score for maximum of 7 mistmatch is  99.86202667484287
Accuracy score for maximum of 8 mistmatch is  99.99233481526905
Accuracy score for maximum of 9 mistmatch is  100.0
